출처 : 핸즈온 머신러닝

# Chapter12 텐서플로를 사용한 사용자 정의 모델과 훈련 

## 12.1 텐서플로 훑어보기 

다양한 API가 존재하여 이에 대해 자세히 살펴보아야 한다고 한다. 

1) 저수준 딥러닝 API
- ```tf.nn```
- ```tf.losses```
- ```tf.metrics```
- ```tf.optimizers```
- ```tf.train```
- ```tf.initializers```

2) 자동미분
- ```tf.GradientTape```
- ```tf.gradients()```

3) 입출력과 전처리
- ```tf.data```
- ```tf.feature_column```
- ```tf.audio```
- ```tf.image```
- ```tf.io```
- ```tf.queue```

4) 텐서보드 시각화
- ```tf.summary```

5) 배포와 최적화
- ```tf.distribute```
- ```tf.saved_model```
- ```tf.autograph```
- ```tf.graph_util```
- ```tf.lite```
- ```tf.quantizaion```
- ```tf.tpu```
- ```tf.xla```

6) 특수한 데이터 구조
- ```tf.lookup```
- ```tf.nest```
- ```tf.ragged```
- ```tf.sets```
- ```tf.sparse```
- ```tf.strings```

7) 선형대수와 신호처리를 포함한 수학연산
- ```tf.math```
- ```tf.linalg```
- ```tf.signal```
- ```tf.random```
- ```tf.bitwise```

8) 그 외 
- ```tf.compat```
- ```tf.config```

## 12.2 넘파이처럼 텐서플로 사용하기

### 12.2.1 텐서와 연산

- tf.constant()  
텐서를 만들어준다.

In [2]:
import tensorflow as tf

In [7]:
#하나의 list가 행이 되는 것 같다.
t = tf.constant([[1.,2.,3.], [4.,5.,6.]])

In [8]:
t.shape

TensorShape([2, 3])

In [9]:
t.dtype

tf.float32

- 다양한 연산 가능 

In [11]:
#모든 행과 1열부터 마지막 열까지만 뽑아라
t[:,1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [17]:
#tf.newaxis는 원래 차원을 변경해주는 것. 근데 여기서는 적용이 안 되는 듯 싶다.
t[...,1,tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [24]:
print(t + 10)
#근데 위 식은 아래 식과 같다.
print(tf.add(t,10))

tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)


In [21]:
#단순히 해당 원소의 제곱값을 찾아준다.
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [27]:
#파이썬 3.5부터 행렬곱은 @으로 연산한다고 한다.
print(t @ tf.transpose(t))

#이는 tf.matmul을 사용하는 것과 같다.
print(tf.matmul(t, tf.transpose(t)))

tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)


- 넘파이와의 차이점

> ```tf.add``` ```tf.multiply``` ```tf.square``` ```tf.exp``` ```tf.sqrt``` ```tf.reshape``` ```tf.squeeze``` ```tf.tile```등은 넘파이 연산과 같음

> ```tf.reduce_mean``` = ```np.mean``` // ```tf.reduce_sum```=```np.sum``` // ```tf.reduce_max```=```np.max``` // ```tf.math.log```=```np.log``` 임

### 12.2.2 텐서와 넘파이

- 텐서와 넘파이는 함께 사용하기 편리함

In [37]:
import numpy as np
import tensorflow as tf

a = np.array([2., 4.,5.])

#넘파이 행렬을 텐서로 만들어준다.
print(tf.constant(a))
print(tf.constant(a).dtype)
print('\n')

#텐서를 넘파이화 해준다.
print(t.numpy()) #또는 np.array(t)
print(t.numpy().dtype)
print('\n')

#텐서 연산에 넘파이 행렬을 넣어줌
print(tf.square(a))
print(tf.square(a).dtype)
print('\n')

#넘파이 연산에 텐서를 넣어줌
print(np.square(t))
print(np.square(t).dtype)

tf.Tensor([2. 4. 5.], shape=(3,), dtype=float64)
<dtype: 'float64'>


[[1. 2. 3.]
 [4. 5. 6.]]
float32


tf.Tensor([ 4. 16. 25.], shape=(3,), dtype=float64)
<dtype: 'float64'>


[[ 1.  4.  9.]
 [16. 25. 36.]]
float32


### 12.2.3 타입변환

- 타입변환은 성능을 크게 감소시킬 수 있다(...!!!!)
- 텐서플로는 이를 방지하기 위해 어떠한 타입 변환도 자동으로 수행하지 않는다.
- 실수 텐서와 정수 텐서를 더할 수도 없고, 32비트 실수와 64비트 실수도 더할 수 없다.

In [39]:
tf.constant(2.) + tf.constant(40)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2]

In [41]:
tf.constant(2.) + tf.constant(40., dtype = tf.float64)


InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:AddV2]

In [43]:
tf.constant(2.) + tf.constant(40., dtype = tf.float32) #이렇게 하면 진행이 된다!

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

- tf.cast()  
타입 변환을 수행한다.

In [45]:
t2 = tf.constant(40., dtype = tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32) #64비트를 32비트로 바꾸어 연산을 진행한다.

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

### 12.2.4 변수

tf.Tensor는 변경이 불가능한 객체이다.  
따라서, 이와 같은 일반적인 텐서로는 역전파로 변경되어야 하는 신경망의 가중치를 구할 수 없다.  
또한 시간에 따라 변경되어야 하는 모멘텀 옵티마이저의 과거 그레디언트 등을 변경할 수도 없다.  
따라서 tf.Variable이 사용된다.

In [52]:
v = tf.Variable([[1.,2.,3.], [4.,5.,6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

- assign 메서드  
해당 메서드를 통해 변수를 증가시키거나 감소시킬 수 있다.  

In [56]:
#v의 값을 두배로 하여 v를 바꾼다.
#이 방식은 조금 조심해야겠는게, 여러 번 실행을 하면 계속해서 바뀌게 된다.
v.assign(2 * v)


<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 4., 84., 12.],
       [16., 20., 24.]], dtype=float32)>

In [57]:
#(0,1)의 값을 42로 바꿈
v[0,1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 4., 42., 12.],
       [16., 20., 24.]], dtype=float32)>

In [61]:
#2열을 [0,1]로 바꾼다.
v[:,2].assign([0.,1.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 4., 42.,  0.],
       [16., 20.,  1.]], dtype=float32)>

In [63]:
#원하는 위치의 값을 따로 바꿀 수 있다.
#이 메서드의 경우 (0,0)를 100으로 바꾸고, (1,2)를 200으로 바꾼다.
v.scatter_nd_update(indices = [[0,0], [1,2]], updates= [100., 200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [ 16.,  20., 200.]], dtype=float32)>

### 12.2.5 다른 데이터 구조

- 희소 텐서(tf.SparseTensor)  
: 대부분 0으로 채워진 텐서를 효율적으로 나타냄.
- 텐서 배열(tf.TensorArray)  
: 텐서의 리스트. 리스트에 포함된 모든 텐서는 크기와 데이터 타입이 동일해야 함.  
- 래그드 텐서(tf.RaggedTensor)  
: 리스트의 리스트를 나타낸다.  
- 문자열 텐서(tf.string)  
: 바이트 문자열을 나타내는데, 유니코드 문자열을 사용해 문자열 텐서를 만들면 자동으로 UTF-8로 인코딩 됨.  
- 집합(set)  
: 집합은 일반적인 텐서로 나타낸다. (아무래도 내부의 값을 변경할 수 없으니 집합으로 인식하는 것인가?)  
- 큐(tf.queue)  
: 큐는 단계별로 텐서를 저장한다. 간단한 FIFO(First In, First Out), 어떤 원소에 우선권을 주는 큐(PriorityQueue), 원소를 섞는 큐(RandomShuffleQueue), 패딩을 추가하여 크기가 다른 원소의 배치를 만드는 큐(PaddingFIFOQueue) 등이 있음.

## 12.3 사용자 정의 모델과 훈련 알고리즘

### 12.3.1 사용자 정의 손실 함수

회귀 모델을 훈련하는 데, 훈련 세트의 이상치가 있는 경우? --> 후버 손실함수를 사용하여, 일부 임계치에 대해서는 제곱을 곱하지 않음

In [80]:
def huber_fn(y_true, y_pred):
    #실제값과 예측값의 차를 만듦
    error = y_true - y_pred
    
    #1보다 작은 에러는 제곱을 적용하기 위해 논리값을 만듦.
    is_small_error = tf.abs(error) < 1
    
    #loss를 계산한다.
    squared_loss = tf.square(error)/2
    linear_loss = tf.abs(error) - 0.5

    return tf.where(is_small_error, squared_loss, linear_loss)
    

=> 특히 이렇게 샘플마다 하나의 손실을 담은 텐서를 반환하는 것이 좋음. 이래야 클래스 가중치나 샘플 가중치를 적용할 수 있음.

> ```tf.where``` : tf.where(condition,a,b) condition이 참이면 a를 참이 아니면 b를 추출함.

In [82]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import numpy as np

#데이터 적재
housing = fetch_california_housing()

#train test 데이터 분할
x_train_full, x_test, y_train_full, y_test = train_test_split(housing.data, housing.target)

#train - valid 데이터 분할
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full)

#표준화 작업을 거쳐주는 것 같다.
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_valid = scaler.fit_transform(x_valid)
x_test = scaler.fit_transform(x_test)

In [86]:
#input 객체 생성 - shape과 dtype을 이용하여 input을 구축
#tabular 데이터에서 필요한 건 결국 칼럼의 수! 해당 obs의 개수는 계속 달라질 수 있으니까
input_ = keras.layers.Input(shape = x_train.shape[1:])

#Dense층 생성
hidden1 = keras.layers.Dense(30, activation = 'relu')(input_)
hidden2 = keras.layers.Dense(30, activation = 'relu')(hidden1) #뒤에 이렇게 함수처럼 넣을 객체를 지정해준다.
#함수처럼이 무슨 의미냐면 sum()처럼 만들자마자 위의 값을 함수처럼 넣는다는 의미

#concatenate층으로 *두번째 은닉층과 input층을 연결해준다.*
concat = keras.layers.Concatenate()([input_, hidden2])
#이게 왜 생긴거냐면 와이드 경로는 입력의 일부 또는 전체가 바로 출력층으로 전달! 그래서 와이드 신경망을 구축하기 위해 넣은 것이라 생각하면 된다.

#출력층 만들기 - concat을 받는다는 것은 input의 값과 hidden2의 출력값을 여기서 출력해준다는 의미다.
output = keras.layers.Dense(1)(concat)

#input과 output을 만들어 케라스 모델을 구축한다
model = keras.Model(inputs = [input_], outputs = [output])


In [87]:
model.compile(loss = huber_fn, optimizer = 'nadam')

In [89]:
model.fit(x_train, y_train, epochs = 30,
                     validation_data = (x_valid, y_valid), verbose = 1)

Epoch 1/30
363/363 [==============================] - 2s 4ms/step - loss: 0.1716 - val_loss: 1.1219
Epoch 2/30
363/363 [==============================] - 2s 5ms/step - loss: 0.1663 - val_loss: 1.1425
Epoch 3/30
363/363 [==============================] - 2s 5ms/step - loss: 0.1635 - val_loss: 1.3806
Epoch 4/30
363/363 [==============================] - 2s 5ms/step - loss: 0.1591 - val_loss: 1.4932
Epoch 5/30
363/363 [==============================] - 2s 5ms/step - loss: 0.1547 - val_loss: 1.6400
Epoch 6/30
363/363 [==============================] - 2s 5ms/step - loss: 0.1521 - val_loss: 1.6977
Epoch 7/30
363/363 [==============================] - 2s 5ms/step - loss: 0.1486 - val_loss: 1.8556
Epoch 8/30
363/363 [==============================] - 2s 5ms/step - loss: 0.1467 - val_loss: 1.7740
Epoch 9/30
363/363 [==============================] - 2s 5ms/step - loss: 0.1441 - val_loss: 1.8068
Epoch 10/30
363/363 [==============================] - 2s 5ms/step - loss: 0.1428 - val_loss: 1.9615

In [98]:
model.save("/model/my_model_with_a_custom_loss.h5")

### 12.3.2 사용자 정의 요소를 가진 모델을 저장하고 로드하기 

<span style="color:yellow">노란 모델을 로드할 때는 함수 이름과 실제 함수를 매핑한 딕셔너리를 전달해야 한다!</span>

In [100]:
model_loaded = keras.models.load_model("/model/my_model_with_a_custom_loss.h5"
                                      ,custom_objects = {"huber_fn" : huber_fn})

- 다른 기준이 필요할 경우?

In [103]:
def create_huber(threshold = 1.0) : 
    def huber_fn(y_true, y_pred) : 
        
        #실제값과 예측값의 차를 만듦.
        error = y_true - y_pred
    
        #1보다 작은 에러는 제곱을 적용하기 위해 논리값을 만듦.
        is_small_error = tf.abs(error) < threshold
    
        #loss를 계산한다.
        squared_loss = tf.square(error)/2
        linear_loss = treshold * tf.abs(error) - threshold**2 / 2

        return tf.where(is_small_error, squared_loss, linear_loss)
    

In [107]:
#이젠 treshold에 따라서 가변적인 손실함수를 구축한다.
model_loaded.compile(loss = create_huber(2.0), optimizer = 'nadam')
model_loaded.save("/model/my_model_with_a_custom_loss_ver1_0.h5")

- 문제는 모델을 저장할 때 이 treshhold 값은 저장되지 않는다. 따라서 모델을 불러올때 treshold 값을 지정해야 한다. 

In [108]:
model_loaded2 = keras.models.load_model("/model/my_model_with_a_custom_loss_ver1_0.h5"
                                       ,custom_objects = {'huber_fn' : create_huber(2.0)})

- class 상속으로 문제를 해결할 수 있을까?

> 우선 기본적으로 keras.losses.Loss는 아래와 같이 생겼다.

Loss base class.

To be implemented by subclasses:
* `call()`: Contains the logic for loss calculation using `y_true`, `y_pred`.

Example subclass implementation:

```python
class MeanSquaredError(Loss):

  def call(self, y_true, y_pred):
    y_pred = tf.convert_to_tensor_v2(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    return tf.reduce_mean(math_ops.square(y_pred - y_true), axis=-1)
```

When used with `tf.distribute.Strategy`, outside of built-in training loops
such as `tf.keras` `compile` and `fit`, please use 'SUM' or 'NONE' reduction
types, and reduce losses explicitly in your training loop. Using 'AUTO' or
'SUM_OVER_BATCH_SIZE' will raise an error.

Please see this custom training [tutorial](
  https://www.tensorflow.org/tutorials/distribute/custom_training) for more
details on this.

You can implement 'SUM_OVER_BATCH_SIZE' using global batch size like:
```python
with strategy.scope():
  loss_obj = tf.keras.losses.CategoricalCrossentropy(
      reduction=tf.keras.losses.Reduction.NONE)
  ....
  loss = (tf.reduce_sum(loss_obj(labels, predictions)) *
          (1. / global_batch_size))
```
Init docstring:
Initializes `Loss` class.

Args:
  reduction: (Optional) Type of `tf.keras.losses.Reduction` to apply to
    loss. Default value is `AUTO`. `AUTO` indicates that the reduction
    option will be determined by the usage context. For almost all cases
    this defaults to `SUM_OVER_BATCH_SIZE`. When used with
    `tf.distribute.Strategy`, outside of built-in training loops such as
    `tf.keras` `compile` and `fit`, using `AUTO` or `SUM_OVER_BATCH_SIZE`
    will raise an error. Please see this custom training [tutorial](
      https://www.tensorflow.org/tutorials/distribute/custom_training)
    for more details.
  name: Optional name for the op.

In [109]:
class HuberLoss(keras.losses.Loss) : 
    
    #기본적인 하이퍼파라미터를 **kwargs로 받은 매개변수 값을 부모 클래스의 생성자에게 전달한다.
    #treshold의 초기값은 1.0으로 설정한다
    def __init__(self, threshold = 1.0, **kwargs) : 
        self.threshold = threshold #treshold가 들어오면 이 값을 설정하면 됨.
        super().__init__(**kwargs) #나머지 항목들은 필요하다면 넣어준다.
        
    #call() 메서드는 레이블과 예측을 받고 샘플의 손실을 계산하여 반환한다.    
    def call(self, y_true, y_pred) : 
        error = y_true - y_pred
    
        #1보다 작은 에러는 제곱을 적용하기 위해 논리값을 만듦.
        is_small_error = tf.abs(error) < threshold
    
        #loss를 계산한다.
        squared_loss = tf.square(error)/2
        linear_loss = treshold * tf.abs(error) - threshold**2 / 2

        return tf.where(is_small_error, squared_loss, linear_loss)
    
    #하이퍼파리미터 이름과 같이 매핑된 딕셔너리를 반환한다.
    def get_config(self) : 
        base_config = super().get_config() #부모 클래스의 get_config를 호출하고 
        return {**base_config, "treshold" : self.threshold} #반환된 딕셔너리에 새로운 하이퍼파라미터를 추가한다.

In [111]:
keras.losses.Loss

TypeError: get_config() missing 1 required positional argument: 'self'